In [ ]:
from src.data.data import *
from src.orcml import *
from src.plotting import *
from src.utils.graph_utils import *
from src.isorc import *
from src.utils.embeddings import *
from sklearn.manifold import TSNE
import umap
%load_ext autoreload

exp_params = {
    'mode': 'nbrs',
    'n_neighbors': 15,
    'epsilon': None,
    'lda': 0.01,
    'delta': 0.8
}

In [ ]:
n_points = 2500
noise = 6.2
noise_thresh = 2.2

dataset_info = {
    'name': '3D_swiss_roll',
    'n_points': n_points,
    'noise': noise,
    'noise_thresh': noise_thresh
}

return_dict = swiss_roll(n_points=n_points, noise=noise, noise_thresh=noise_thresh, supersample=True, dim=3, hole=False)
swiss_roll_data, color, cluster, swiss_roll_supersample, subsample_indices = return_dict['data'], return_dict['color'], return_dict['cluster'], return_dict['data_supersample'], return_dict['subsample_indices']

In [ ]:
orcmanl = ORCManL(
    exp_params=exp_params,
    verbose=True,
    reattach=True
)
orcmanl.fit(return_dict['data'])

In [ ]:
plot_graph_3D(return_dict['data'], orcmanl.G_pruned, title=None)

In [ ]:
%autoreload 2
import gc
torch.cuda.empty_cache()
gc.collect()

isorc = ISORC(orcmanl, dim=2, init='spectral')
X_opt, losses = isorc.fit_graph(lr=0.5, n_iter=1500)

plt.plot(losses)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()

In [ ]:
plot_graph_2D(X_opt, orcmanl.G, title=None, node_color=return_dict['color'][orcmanl.G.nodes()])

In [ ]:
# quadratics
n_points = 2000
noise = 0.20
noise_thresh = 0.45

return_dict = quadratics(n_points=n_points, noise=noise, noise_thresh=noise_thresh, supersample=True)

In [ ]:
orcmanl = ORCManL(
    exp_params=exp_params,
    verbose=True,
    reattach=True
)
orcmanl.fit(return_dict['data'])

In [ ]:
plot_graph_2D(return_dict['data'], orcmanl.G_pruned, title=None)

In [ ]:
%autoreload 2
import gc
torch.cuda.empty_cache()
gc.collect()

isorc = ISORC(orcmanl, dim=2, init='spectral')
X_opt, losses = isorc.fit_graph(lr=0.2, n_iter=500)

plt.plot(losses)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()

In [ ]:
%autoreload 2
plot_graph_2D(X_opt, orcmanl.G, title=None, node_color=return_dict['cluster'][orcmanl.G.nodes()])

In [ ]:
# quadratics # more noise
n_points = 2000
noise = 0.20
noise_thresh = 0.5

return_dict = quadratics(n_points=n_points, noise=noise, noise_thresh=noise_thresh, supersample=True)

In [ ]:
orcmanl = ORCManL(
    exp_params=exp_params,
    verbose=True,
    reattach=True
)
orcmanl.fit(return_dict['data'])

In [ ]:
plot_graph_2D(return_dict['data'], orcmanl.G_pruned, title=None)

In [ ]:
%autoreload 2
import gc
torch.cuda.empty_cache()
gc.collect()

isorc = ISORC(orcmanl, dim=2, init='spectral')
X_opt, losses = isorc.fit_graph(lr=0.2, n_iter=500, weight_orc=True, p=10)

plt.plot(losses)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()

In [ ]:
%autoreload 2
plot_graph_2D(X_opt, orcmanl.G, title=None, node_color=return_dict['cluster'][orcmanl.G.nodes()], edge_color=orcmanl.orcs)

In [ ]:
print(isorc.W[isorc.W != 0])